# Reading netCDF data
- requires [numpy](http://numpy.scipy.org) and netCDF/HDF5 C libraries.
- Github site: https://github.com/Unidata/netcdf4-python
- Online docs: http://unidata.github.io/netcdf4-python/
- Based on Konrad Hinsen's old [Scientific.IO.NetCDF](http://dirac.cnrs-orleans.fr/plone/software/scientificpython/) API, with lots of added netcdf version 4 features.
- Developed by Jeff Whitaker at NOAA, with many contributions from users.

## Interactively exploring a netCDF File

Let's explore a netCDF file from the *Atlantic Real-Time Ocean Forecast System*

first, import netcdf4-python and numpy

In [3]:
import netCDF4
import numpy as np

## Create a netCDF4.Dataset object
- **`f`** is a `Dataset` object, representing an open netCDF file.
- printing the object gives you summary information, similar to *`ncdump -h`*.

In [4]:
f = netCDF4.Dataset('/Users/vsood/Downloads/prods_op_mogreps-uk_20140717_03_11_015.nc', 'r')
print(f) 

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    source: Data from Met Office Unified Model
    um_version: 8.5
    Conventions: CF-1.5
    dimensions(sizes): time(36), grid_latitude(548), grid_longitude(421), time_0(3), pressure(3), time_1(3), bnds(2), pressure_0(2), pressure_1(3), dim0(3), grid_longitude_0(421), grid_latitude_0(547), time_3(3)
    variables(dimensions): float32 stratiform_snowfall_rate(time,grid_latitude,grid_longitude), int32 rotated_latitude_longitude(), float64 time(time), float32 grid_latitude(grid_latitude), float32 grid_longitude(grid_longitude), float64 forecast_period(time), float64 forecast_reference_time(), float32 cloud_base_altitude_assuming_only_consider_cloud_area_fraction_greater_than_2p5_oktas(time_0,grid_latitude,grid_longitude), float64 time_0(time_0), float64 forecast_period_0(time_0), float32 cloud_area_fraction_assuming_maximum_random_overlap(time_0,grid_latitude,grid_longitude), float32 wet_bulb_freezing_

## Access a netCDF variable
- variable objects stored by name in **`variables`** dict. A variable typically depends on one or more ***`dimensions`*** making them multi-dimensional
- print the variable yields summary info (including all the attributes).
- no actual data read yet (just have a reference to the variable object with metadata).

In [22]:
print(f.variables.keys()) # get all variable names
#print(f.variables.values()) # get all variable names
temp = f.variables['air_temperature']  # temperature variable
print(temp) 

odict_keys(['stratiform_snowfall_rate', 'rotated_latitude_longitude', 'time', 'grid_latitude', 'grid_longitude', 'forecast_period', 'forecast_reference_time', 'cloud_base_altitude_assuming_only_consider_cloud_area_fraction_greater_than_2p5_oktas', 'time_0', 'forecast_period_0', 'cloud_area_fraction_assuming_maximum_random_overlap', 'wet_bulb_freezing_level_altitude', 'wet_bulb_potential_temperature', 'pressure', 'unknown', 'air_pressure_at_sea_level', 'air_temperature', 'height', 'air_temperature_0', 'time_1', 'time_1_bnds', 'forecast_period_1', 'forecast_period_1_bnds', 'air_temperature_1', 'air_temperature_2', 'pressure_0', 'dew_point_temperature', 'fog_area_fraction', 'geopotential_height', 'pressure_1', 'high_type_cloud_area_fraction', 'low_type_cloud_area_fraction', 'medium_type_cloud_area_fraction', 'relative_humidity', 'relative_humidity_0', 'specific_humidity', 'stratiform_rainfall_amount', 'forecast_period_2', 'forecast_period_2_bnds', 'time_2', 'time_2_bnds', 'stratiform_rain

## List the Dimensions

- All variables in a netCDF file have an associated shape, specified by a list of dimensions.
- Let's list all the dimensions in this netCDF file.
- Note that the **`time`** dimension is special (*`unlimited`*), which means it can be appended to.

In [10]:
for d1 in f.dimensions.keys():
    print(d1)

time
grid_latitude
grid_longitude
time_0
pressure
time_1
bnds
pressure_0
pressure_1
dim0
grid_longitude_0
grid_latitude_0
time_3


In [14]:
for d2 in f.dimensions.values():
    print(d2)

<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 36

<class 'netCDF4._netCDF4.Dimension'>: name = 'grid_latitude', size = 548

<class 'netCDF4._netCDF4.Dimension'>: name = 'grid_longitude', size = 421

<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time_0', size = 3

<class 'netCDF4._netCDF4.Dimension'>: name = 'pressure', size = 3

<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time_1', size = 3

<class 'netCDF4._netCDF4.Dimension'>: name = 'bnds', size = 2

<class 'netCDF4._netCDF4.Dimension'>: name = 'pressure_0', size = 2

<class 'netCDF4._netCDF4.Dimension'>: name = 'pressure_1', size = 3

<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'dim0', size = 3

<class 'netCDF4._netCDF4.Dimension'>: name = 'grid_longitude_0', size = 421

<class 'netCDF4._netCDF4.Dimension'>: name = 'grid_latitude_0', size = 547

<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time_3', size = 3



Each variable has a **`dimensions`** and a **`shape`** attribute.

In [7]:
temp.dimensions

('time_0', 'grid_latitude', 'grid_longitude')

In [8]:
temp.shape

(3, 548, 421)

### Each dimension typically has a variable associated with it (called a *coordinate* variable).
- *Coordinate variables* are 1D variables that have the same name as dimensions.
- Coordinate variables and *auxiliary coordinate variables* (named by the *coordinates* attribute) locate values in time and space.

In [30]:
time = f.variables['time_0']
long = f.variables['grid_longitude']
lat = f.variables['grid_latitude']
print(time)
print(long)                 

<class 'netCDF4._netCDF4.Variable'>
float64 time_0(time_0)
    axis: T
    units: hours since 1970-01-01 00:00:00
    standard_name: time
    calendar: gregorian
unlimited dimensions: time_0
current shape = (3,)
filling on, default _FillValue of 9.969209968386869e+36 used

<class 'netCDF4._netCDF4.Variable'>
float32 grid_longitude(grid_longitude)
    axis: X
    units: degrees
    standard_name: grid_longitude
unlimited dimensions: 
current shape = (421,)
filling on, default _FillValue of 9.969209968386869e+36 used



## Accessing data from a netCDF variable object

- netCDF variables objects behave much like numpy arrays.
- slicing a netCDF variable object returns a numpy array with the data.
- Boolean array and integer sequence indexing behaves differently for netCDF variables than for numpy arrays. Only 1-d boolean arrays and integer sequences are allowed, and these indices work independently along each dimension (similar to the way vector subscripts work in fortran).

In [56]:
time_value = time[:]  # Reads the netCDF variable MT, array of one element
print(time_value) 

[390448. 390449. 390450.]


In [49]:
long_value = long[:] # examine first 10 values of longitude
print(long_value) 

[354.91074 354.93073 354.95074 354.97073 354.99072 355.01074 355.03073
 355.05075 355.07074 355.09073 355.11075 355.13074 355.15073 355.17075
 355.19073 355.21072 355.23074 355.25073 355.27072 355.29074 355.31073
 355.33075 355.35074 355.37073 355.39075 355.41074 355.43073 355.45074
 355.47073 355.49072 355.51074 355.53073 355.55075 355.57074 355.59073
 355.61075 355.63074 355.65073 355.67075 355.69073 355.71072 355.73074
 355.75073 355.77072 355.79074 355.81073 355.83075 355.85074 355.87073
 355.89075 355.91074 355.93073 355.95074 355.97073 355.99072 356.01074
 356.03073 356.05075 356.07074 356.09073 356.11075 356.13074 356.15073
 356.17075 356.19073 356.21072 356.23074 356.25073 356.27072 356.29074
 356.31073 356.33075 356.35074 356.37073 356.39075 356.41074 356.43073
 356.45074 356.47073 356.49072 356.51074 356.53073 356.55075 356.57074
 356.59073 356.61075 356.63074 356.65073 356.67075 356.69073 356.71072
 356.73074 356.75073 356.77072 356.79074 356.81073 356.83075 356.85074
 356.8

In [50]:
lat_value = lat[:] # examine first 10 values of longitude
print(lat_value) 

[-3.77994990e+00 -3.75994992e+00 -3.73994994e+00 -3.71994996e+00
 -3.69994998e+00 -3.67995000e+00 -3.65995002e+00 -3.63994980e+00
 -3.61994982e+00 -3.59994984e+00 -3.57994986e+00 -3.55994987e+00
 -3.53994989e+00 -3.51994991e+00 -3.49994993e+00 -3.47994995e+00
 -3.45994997e+00 -3.43994999e+00 -3.41995001e+00 -3.39995003e+00
 -3.37995005e+00 -3.35994983e+00 -3.33994985e+00 -3.31994987e+00
 -3.29994988e+00 -3.27994990e+00 -3.25994992e+00 -3.23994994e+00
 -3.21994996e+00 -3.19994998e+00 -3.17995000e+00 -3.15994978e+00
 -3.13994980e+00 -3.11994982e+00 -3.09994984e+00 -3.07994986e+00
 -3.05994987e+00 -3.03994989e+00 -3.01994991e+00 -2.99994993e+00
 -2.97994995e+00 -2.95994997e+00 -2.93994999e+00 -2.91995001e+00
 -2.89995003e+00 -2.87995005e+00 -2.85995007e+00 -2.83994985e+00
 -2.81994987e+00 -2.79994988e+00 -2.77994990e+00 -2.75994992e+00
 -2.73994994e+00 -2.71994996e+00 -2.69994998e+00 -2.67994976e+00
 -2.65994978e+00 -2.63994980e+00 -2.61994982e+00 -2.59994984e+00
 -2.57994986e+00 -2.55994

In [58]:
time_value,long_value, lat_value = time[:],long[:],lat[:]
print(long_value[0:1], lat_value[0:1], time_value[0:1])
#tempslice = temp[0, dpth > 400, yy > yy.max()/2, xx > xx.max()/2]
#print('shape of temp slice: %s' % repr(tempslice.shape))

[354.91074] [-3.77995] [390448.]


In [61]:
t, l, la = long_value[0:1], lat_value[0:1], time_value[0:1]

In [62]:
print(t)

[354.91074]


In [63]:
print(l)

[-3.77995]


In [64]:
print(la)

[390448.]


In [68]:
print(temp)

<class 'netCDF4._netCDF4.Variable'>
float32 air_temperature(time_0, grid_latitude, grid_longitude)
    _FillValue: -1073741800.0
    standard_name: air_temperature
    units: K
    um_stash_source: m01s03i236
    grid_mapping: rotated_latitude_longitude
    coordinates: forecast_period_0 forecast_reference_time height
unlimited dimensions: time_0
current shape = (3, 548, 421)
filling on
